<a href="https://colab.research.google.com/github/wallik2/Jobsdb_WebScraper/blob/main/JobsDB_Webscraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install tqdm

In [2]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd, numpy as np
import re
import csv
from tqdm.notebook import tqdm

from google.colab import data_table
data_table.enable_dataframe_formatter()


print("Setup completed")

Setup completed


In [3]:
pd.set_option('display.max_colwidth', 20)






---



Enter your keyword to search here 

In [4]:
Job_keyword = 'doctor'               #@param {type: "string"}

In [5]:
url = f'https://th.jobsdb.com/th/search-jobs/{Job_keyword}/'

#Get source code
html_text = requests.get(url).text



---



In [6]:
# whole page  # convert to soup object
soup = BeautifulSoup(html_text,'lxml')

In [7]:
# jobs object
jobs = soup.find_all(class_="sx2jih0 zcydq852 zcydq842 zcydq872 zcydq862 zcydq82a zcydq832 zcydq8d2 zcydq8cq")

<details>
  <summary>HTML of Job title and Job company</summary>

                        <div class="sx2jih0 zcydq89i">
                         <h1 class="sx2jih0 zcydq82q _18qlyvc0 _18qlyvcv _18qlyvc3 _18qlyvc8">
                          <a class="DvvsL_0 _1p9OP" href="/th/en/job/data-scientist-300003002440231?token=0~f866067f-8b1f-4bb8-80b1-1c411802dd0a&amp;sectionRank=31&amp;jobId=jobsdb-th-job-300003002440231" rel="nofollow noopener noreferrer" target="_top">
                           <div class="sx2jih0 _2j8fZ_0 sIMFL_0 _1JtWu_0">
                            Data Scientist
                           </div>
                          </a>
                         </h1>
                         <span class="sx2jih0 zcydq82q _18qlyvc0 _18qlyvcv _18qlyvc1 _18qlyvc8">
                          Donuts Bangkok Co., Ltd.
                         </span>
                        </div>

In [8]:
# check the source code if you want to

#print(soup.prettify())

In [9]:
# Find the page number available

page_number = soup.find_all(class_="sx2jih0 sx2jih1 sx2jih6 sx2jih8 zcydq8r zcydq82c zcydq87e zcydq83q zcydq815 tfxl6i0 _3dxvbx0 _3dxvbx1 _18qlyvc0 _18qlyvcv _18qlyvc1 _18qlyvc8 _18qlyvc11")
page_number = re.findall(r'\d+(?=</option></select>)',str(page_number[0]))

page_number = int(page_number[0])

print(page_number)

2




---



Jobs object have 2 types : have logo, and have no logo,

They have different class name

We store the data in csv file

In [10]:
csv_file = open('job_records.csv','w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['job title','more info','job_company','job_location','skill requirement 1','skill requirement 2','skill requirement 3','posted time'])

118

In [11]:
# Object for each job 

def scrping(jobs,csv_file): 


  for index, job in enumerate(jobs):

    job_title = job.find('h1',class_='sx2jih0 zcydq82q _18qlyvc0 _18qlyvcv _18qlyvc3 _18qlyvc8').text
    more_info = job.h1.a['href']
    url_more_info = f'https://th.jobsdb.com/{more_info}'

    #for the company that have logo
    try:
      job_company = job.find('span',class_ ='sx2jih0 zcydq82q _18qlyvc0 _18qlyvcv _18qlyvc1 _18qlyvc8').text

    #for the company that doesn't have logo  
    except:
      job_company = job.find('span',class_ ='sx2jih0 zcydq8s zcydq82u').text
    
    job_location = job.find('span', class_ = 'sx2jih0 zcydq82q _18qlyvc0 _18qlyvcv _18qlyvc3 _18qlyvc6').text
    job_time = job.find('span', class_ = 'sx2jih0 zcydq82q _18qlyvc0 _18qlyvcx _18qlyvc1 _18qlyvc6').text
    job_description_uls = job.find('ul', class_ = 'sx2jih0 sx2jih3 h6p8rp0 h6p8rp5')
  

    # append all skill requirement for each 
    skill_requirement = []
    for ul in job_description_uls:
      skill_requirement.append(ul.text)
  
    # Skill requirement have less than 3
    if len(skill_requirement) != 3:

      for i in range(3-len(skill_requirement)):
        skill_requirement.append(np.nan)


    #write csv_file 
    csv_writer.writerow([job_title,url_more_info,job_company,job_location,skill_requirement[0],skill_requirement[1],skill_requirement[2],job_time])

  return csv_file


In [12]:
#@title Future project (Forbidden) {display-mode: "form"}

"""

i=1
while i == 1: 
  df = scrping(jobs,df)
  next = soup.find(class_='sx2jih0 zcydq872 zcydq862 zcydq88 zcydq82b zcydq832 zcydq8c6 zcydq824 zcydq82l zcydq82k _1ouuf_0')['href']
  #print(next)
  url2 = f'https://th.jobsdb.com/{next}'

  html_text2 = requests.get(url2).text
  soup2 = BeautifulSoup(html_text2,'lxml')
  jobs = soup2.find_all(class_="sx2jih0 zcydq852 zcydq842 zcydq872 zcydq862 zcydq82a zcydq832 zcydq8d2 zcydq8cq")

"""

'\n\ni=1\nwhile i == 1: \n  df = scrping(jobs,df)\n  next = soup.find(class_=\'sx2jih0 zcydq872 zcydq862 zcydq88 zcydq82b zcydq832 zcydq8c6 zcydq824 zcydq82l zcydq82k _1ouuf_0\')[\'href\']\n  #print(next)\n  url2 = f\'https://th.jobsdb.com/{next}\'\n\n  html_text2 = requests.get(url2).text\n  soup2 = BeautifulSoup(html_text2,\'lxml\')\n  jobs = soup2.find_all(class_="sx2jih0 zcydq852 zcydq842 zcydq872 zcydq862 zcydq82a zcydq832 zcydq8d2 zcydq8cq")\n\n'

                        <div class="sx2jih0 zcydq89i">
                         <h1 class="sx2jih0 zcydq82q _18qlyvc0 _18qlyvcv _18qlyvc3 _18qlyvc8">
                          <a class="DvvsL_0 _1p9OP" href="/th/en/job/data-scientist-300003002440231?token=0~f866067f-8b1f-4bb8-80b1-1c411802dd0a&amp;sectionRank=31&amp;jobId=jobsdb-th-job-300003002440231" rel="nofollow noopener noreferrer" target="_top">
                           <div class="sx2jih0 _2j8fZ_0 sIMFL_0 _1JtWu_0">
                            Data Scientist
                           </div>
                          </a>
                         </h1>
                         <span class="sx2jih0 zcydq82q _18qlyvc0 _18qlyvcv _18qlyvc1 _18qlyvc8">
                          Donuts Bangkok Co., Ltd.
                         </span>
                        </div>

In [13]:
#MAIN SCRAPER

pn = 1
print(f"There are {page_number} pages for keyword '{Job_keyword}'",'\n')
p_bar = tqdm(range(page_number))
#print(f"There are {page_number} pages for keyword '{Job_keyword}'")



#for i in p_bar:
for i in p_bar:
  
  p_bar.set_description("Processing")

  #print(f'> page:{pn}/{page_number}')  
  
  csv_file = scrping(jobs,csv_file)
  
  pn += 1
  url2 =  f'https://th.jobsdb.com/th/search-jobs/{Job_keyword}/{pn}'

  

  html_text2 = requests.get(url2).text
  soup2 = BeautifulSoup(html_text2,'lxml')
  jobs = soup2.find_all(class_="sx2jih0 zcydq852 zcydq842 zcydq872 zcydq862 zcydq82a zcydq832 zcydq8d2 zcydq8cq")

  

There are 2 pages for keyword 'doctor' 



  0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
df = pd.read_csv('job_records.csv',error_bad_lines=False)

In [15]:
# Download to your local

'''

from google.colab import files

files.download('job_records.csv')

'''

"\n\nfrom google.colab import files\n\nfiles.download('job_records.csv')\n\n"

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
df.to_csv(f'/content/drive/MyDrive/Jobsdb/keyword: {Job_keyword}.csv')

In [18]:
df['more info'] = df['more info'].str[:25]

In [19]:
df

,job title,more info,job_company,job_location,skill requirement 1,skill requirement 2,skill requirement 3,posted time
0,Medical Director...,https://th.jobsd...,Demarest Ltd,Phuket,NaN,NaN,NaN,3d ago
1,Medical Advisor,https://th.jobsd...,360 Degree Searc...,Bangkok,Senior Medical A...,Leading Internat...,Attractive Remun...,2d ago
2,Regulatory Affai...,https://th.jobsd...,WINNERGY MEDICAL...,Wangthonglang,มีโอกาสที่จะได้ร...,ส่งเสริมการพัฒนา...,นโยบายทำงาน 5 วั...,10h ago
3,Staff- Health Cl...,https://th.jobsd...,Krungthai-AXA Li...,Huaykwang,Degree in Nursing,2 years' experie...,"Reviews, analyze...",9h ago
4,แม็คโคร สาขาบางค...,https://th.jobsd...,Siam Makro Publi...,Bangkor-laem,NaN,NaN,NaN,2d ago
5,พนักงานส่วนซ่อมบ...,https://th.jobsd...,Siam Makro Publi...,Nakornnayok,NaN,NaN,NaN,2d ago
6,พนักงานส่วนเบเกอ...,https://th.jobsd...,Siam Makro Publi...,Eastern > Others,NaN,NaN,NaN,2d ago
7,พนักงานขายแผนกซี...,https://th.jobsd...,Terminal Seal an...,Beungkum,NaN,NaN,NaN,6h ago
8,แม็คโคร สาขาสามเ...,https://th.jobsd...,Siam Makro Publi...,Bangkok,NaN,NaN,NaN,2d ago
9,แม็คโคร สาขาประจ...,https://th.jobsd...,Siam Makro Publi...,Prachuap Khiri Khan,NaN,NaN,NaN,2d ago




---



In [20]:
#@title Future project (Get salary) {display-mode: "form"}

# This code will be hidden when the notebook is loaded.
"""
  #Job type zone
  url_more_info = f'https://th.jobsdb.com/{job_description}'
  html_text_more_info = requests.get(url_more_info).text
  soup_more_info = BeautifulSoup(html_text_more_info,'lxml')
  
  jobs = soup.find('span',class_="sx2jih0 zcydq82q _18qlyvc0 _18qlyvcv _18qlyvc1 _1d0g9qk4 _18qlyvc9")
"""

'\n  #Job type zone\n  url_more_info = f\'https://th.jobsdb.com/{job_description}\'\n  html_text_more_info = requests.get(url_more_info).text\n  soup_more_info = BeautifulSoup(html_text_more_info,\'lxml\')\n  \n  jobs = soup.find(\'span\',class_="sx2jih0 zcydq82q _18qlyvc0 _18qlyvcv _18qlyvc1 _1d0g9qk4 _18qlyvc9")\n'